In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install tokenizers
!pip install transformers==2.11.0

     |████████████████████████████████| 3.3MB 8.7MB/s 
     |████████████████████████████████| 675kB 8.7MB/s 
     |████████████████████████████████| 5.6MB 11.1MB/s 
     |████████████████████████████████| 1.2MB 49.4MB/s 
     |████████████████████████████████| 901kB 42.7MB/s 
  Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3


In [ ]:
import sys
sys.path.append("/gdrive/MyDrive/colab/transfer_learn/mrc")
from tokenization_kocharelectra import KoCharElectraTokenizer


In [ ]:
import torch
import numpy as np
import os

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from transformers.configuration_electra import ElectraConfig
from transformers.modeling_electra import ElectraPreTrainedModel, ElectraModel
from transformers.optimization import AdamW

from tokenization_kocharelectra import KoCharElectraTokenizer

class ElectraMRC(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        # 분류할 라벨의 개수
        self.num_labels = config.num_labels

        # ELECTRA 모델
        self.electra = ElectraModel(config)

        # Span 범위 예측을 위한 linear
        self.projection_layer = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        # input_ids, attention_mask, token_type_ids 형태: [batch, seq_len]
        # electra_outputs 형태: [1, batch, seq_len, hidden]
        electra_outputs = self.electra(input_ids, attention_mask, token_type_ids)
 
        # hypothesis 형태: [batch, seq_len, 2 (start/end)]
        hypothesis = self.projection_layer(electra_outputs[0])

        # start, end 형태: [batch, seq_len, 1] -> [batch, seq_len]
        p_start, p_end = hypothesis.split(1, dim=-1)
        p_start = p_start.squeeze(-1)
        p_end = p_end.squeeze(-1)
 
        return p_start, p_end

In [ ]:

def convert_data2feature(config, input_sequence, tokenizer):
    # input_sequence : [CLS] Question [SEP] Context [SEP]
    # => [CLS] 세 종 대 왕 은 _ 몇 _ 대 _ 왕 인 가 ? [SEP] 세 종 대 왕 은 _ 조 선 의 _ ~ [SEP]"

    # 고정 길이 벡터 생성
    input_ids = np.zeros(config["max_length"], dtype=np.int)
    attention_mask = np.zeros(config["max_length"], dtype=np.int)
    segment_ids = np.zeros(config["max_length"], dtype=np.int)

    is_context = False
    for idx, token in enumerate(input_sequence.split()):
        input_ids[idx] = tokenizer._convert_token_to_id(token)
        attention_mask[idx] = 1
        if is_context:
            segment_ids[idx] = 1
        if token == '[SEP]':
            is_context = True

    return input_ids, attention_mask, segment_ids

# 데이터 읽기 함수
def read_data(file_path, tokenizer):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    # 데이터를 저장하기 위한 리스트 생성
    all_input_ids, all_attention_mask, all_segment_ids, start_indexes, end_indexes = [], [], [], [], []
    for idx, line in enumerate(lines):
        input_sequence, start_idx, end_idx = line.strip().split("\t")
        input_ids, attention_mask, segment_ids = convert_data2feature(config, input_sequence, tokenizer)

        all_input_ids.append(input_ids)
        all_attention_mask.append(attention_mask)
        all_segment_ids.append(segment_ids)
        start_indexes.append(int(start_idx))
        end_indexes.append(int(end_idx))

    all_input_ids = torch.tensor(all_input_ids, dtype=torch.long)
    all_attention_mask = torch.tensor(all_attention_mask, dtype=torch.long)
    all_segment_ids = torch.tensor(all_segment_ids, dtype=torch.long)
    start_indexes = torch.tensor(start_indexes, dtype=torch.long)
    end_indexes = torch.tensor(end_indexes, dtype=torch.long)

    return all_input_ids, all_attention_mask, all_segment_ids, start_indexes, end_indexes


In [ ]:

def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

def do_test(model, tokenizer):
    # 평가 모드 셋팅
    model.eval()

    # 평가 데이터 Load
    all_input_ids, all_attention_mask, all_segment_ids, start_indexes, end_indexes = \
        read_data(tokenizer=tokenizer, file_path=config["test_data_path"])

    # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
    test_features = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, start_indexes, end_indexes)
    test_dataloader = DataLoader(test_features, shuffle = True, batch_size=1)

    for step, batch in enumerate(test_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, attention_mask, segment_ids, a_start, a_end = batch

        # 입력 데이터에 대한 출력과 loss 생성
        # p_start, p_end 형태:[1, seq_len]
        p_start, p_end = model(input_ids, attention_mask, segment_ids)
  
        p_start = p_start.argmax(dim=-1)
        p_start = tensor2list(p_start)[0]
        p_end = p_end.argmax(dim=-1)
        p_end_ = tensor2list(p_end)[0]

        a_start = tensor2list(a_start)[0]
        a_end = tensor2list(a_end)[0]

        # 입력 Text 생성
        input_token_ids = tensor2list(input_ids)[0]
        input_tokens = [tokenizer._convert_id_to_token(e) for e in input_token_ids]

        # 입력 Text에서 예측/정답 Span 추출
        predict_span = input_tokens[p_start:p_end+1]
        answer_span = input_tokens[a_start:a_end+1]

        # 입력 Seqquence의 질문, 단락 위치 저장
        segment_positions = [position for position, token in enumerate(input_tokens) if token == "[SEP]"]

        # 모델 예측 확인
        if step < 5:
            question = ''.join(input_tokens[1:segment_positions[0]]).replace("_", " ")
            context = ''.join(input_tokens[segment_positions[0] + 1:segment_positions[1]]).replace("_", " ")
            print("\n\n######################################")
            print("Context : ", context)
            print("Question : ", question)
            print("Answer Span : ", ''.join(predict_span).replace("_", " "))
            print("Predict Span : ", ''.join(answer_span).replace("_", " "))
        else:
            break

def test(config):
    # electra config 객체 생성
    electra_config = ElectraConfig.from_pretrained(
        os.path.join(config["output_dir"], "checkpoint-{0:d}".format(config["checkpoint"])),
        num_labels=config["num_labels"])

    # electra tokenizer 객체 생성
    electra_tokenizer = KoCharElectraTokenizer.from_pretrained(
        os.path.join(config["output_dir"], "checkpoint-{0:d}".format(config["checkpoint"])),
        do_lower_case=False)

    # electra model 객체 생성
    model = ElectraMRC.from_pretrained(
        os.path.join(config["output_dir"], "checkpoint-{0:d}".format(config["checkpoint"])),
        config=electra_config).cuda()

    do_test(model=model, tokenizer=electra_tokenizer)


In [ ]:

def train(config):
    # electra config 객체 생성
    electra_config = ElectraConfig.from_pretrained("monologg/kocharelectra-base-discriminator",
                                                   num_labels=config["num_labels"])
    # electra tokenizer 객체 생성
    electra_tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-base-discriminator",
                                                               do_lower_case=False)
    # electra model 객체 생성
    model = ElectraMRC.from_pretrained("monologg/kocharelectra-base-discriminator",
                                                                     config=electra_config).cuda()
    # 데이터 읽기
    all_input_ids, all_attention_mask, all_token_type_ids, start_indexes, end_indexes = \
        read_data(tokenizer=electra_tokenizer, file_path=config["train_data_path"])

    # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
    train_features = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, start_indexes, end_indexes)
    train_dataloader = DataLoader(train_features, batch_size=config["batch_size"])

    # 크로스 엔트로피 손실 함수
    loss_func = nn.CrossEntropyLoss()

    # 옵티마이저 함수 지정
    optimizer = AdamW(model.parameters(), lr=config["learning_rate"])

    for epoch in range(config["epoch"]):
        for step, batch in enumerate(train_dataloader):
            # 학습 모드 셋팅
            model.train()
            # batch = (input_ids[step], attention_mask[step], segment_ids[step],
            #                                    start_index[step], end_index[step])*batch_size
            # .cuda()를 통해 메모리에 업로드
            batch = tuple(t.cuda() for t in batch)

            # 변화도를 0으로 변경
            optimizer.zero_grad()

            # p_start, p_end 형식: [batch, seq_len]
            # a_start, a_end 형식: [batch]
            input_ids, attention_mask, segment_ids, a_start, a_end = batch
            p_start, p_end = model(input_ids, attention_mask, segment_ids)

            start_loss = loss_func(p_start, a_start)
            end_loss = loss_func(p_end, a_end)

            total_loss = start_loss + end_loss

            # 손실 역전파 수행
            total_loss.backward()
            optimizer.step()

            # 50 batch_step 마다 Loss 출력
            if (step + 1) % 50 == 0:
                print("Current Step : {0:d} / {1:d}\tCurrent Loss : {2:.4f}".format(step+1, int(len(all_input_ids) / config['batch_size']), total_loss.item()))

            # 500 batch_step 마다 결과 출력
            if (step + 1) % 500 == 0:
                do_test(model, electra_tokenizer)

        # 에폭마다 가중치 저장
        output_dir = os.path.join(config["output_dir"], "checkpoint-{}".format(epoch+1))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        electra_config.save_pretrained(output_dir)
        electra_tokenizer.save_pretrained(output_dir)
        model.save_pretrained(output_dir)


In [ ]:
if __name__ == "__main__":
    root_dir = "/gdrive/MyDrive/colab/transfer_learn/mrc"
    output_dir = os.path.join(root_dir, "output")

    if (not os.path.exists(output_dir)):
        os.makedirs(output_dir)

    config = {"mode": "test",
              "train_data_path": os.path.join(root_dir, "mrc_train.txt"),
              "test_data_path": os.path.join(root_dir, "mrc_dev.txt"),
              "output_dir": output_dir,
              "checkpoint": 3,
              "epoch": 3,
              "learning_rate": 5e-5,
              "batch_size": 16,
              "max_length": 512,
              "num_labels": 2,
              }

    if (config["mode"] == "train"):
        train(config)
    else:
        test(config)



######################################
Context :  2008년 2월 25일 이명박은 취임식과 함께 업무 수행을 시작했다. 새 정부의 이름은 각 정권마다 추구하는 핵심 가치를 담아 정권의 이름을 사용한 전 정부들과는 달리 대통령의 실명을 공식적으로 정권 이름에 사용하게 되었다(이것이 첫 사례였다). 이명박 대통령이 17대 대선 후보로 활동하던 당시에 강조하였던 '창조적 실용주의'를 반영하여 간혹 실용정부([UNK][UNK][UNK][UNK])라는 명칭이 사용되기도 한다. 그리고 인수위원회에서는 작은정부 구축을 위해 정부조직을 대대적으로 통폐합하여 개편안을 발표했다. 주 목표는 '작은 정부, 큰 시장'을 큰 뼈대로 '경제살리기'가 목표였다. 한편 이명박 정부 출범의 뒤를 이어 총선거가 치러졌는데, 이 선거에서 여당인 한나라당이 최다 의석을 차지했다.
Question :  이명박이 취임식과 함께 업무수행을 시작한 해는?
Answer Span :  2008년
Predict Span :  2008년


######################################
Context :  부모는 사건 이후 일을 그만두고 딸의 치료에만 매달렸다. 안산시에서 지원금을 받아 병원비와 각종 경비를 부담하고 있었다. 보험사도 끔찍한 사고를 감안해 4000만 원의 보험금을 지급했다. 그러자 안산시는 시에서 받은 긴급치료지원비 600만 원을 모두 반납하라고 명령하면서 만일 이행하지 않을 경우 전세금을 압류하겠다고 안산시장 명의의 공문을 지난 2009년 6월 발송하였다. 또 생활보호대상자 혜택도 중단한다고 통보했다. “원칙적으로 통장에 300만 원 이상의 잔고가 있으면 지원대상에서 제외된다"는 이유였다. 부모는 딸의 신체 중 일부 기능이 영구 상실됐고 앞으로 몇 년은 더 심리치료를 받아야 한다고 사정했으나 받아들여지지 않았다.
Question :  조두순 사건에서 보험사는 부모에게 얼마를 지급했는가?
Answer Span :  4000만 원